## Production Amounts and Scope 1 and 2 Emission Values Per EAF Plant in the United States

### Loading Libraries

In [40]:
import pandas as pd
import numpy as np
import janitor
import geopandas as gpd
import mapclassify as mc
from shapely.geometry import Point

### Importing eGrid Data

The EPA eGrida Data, which was available for 2021 at the time of download, contains the emissions intensity of electricity produced in each sub region. For this analysis, we assume that each steel plant is drawing 100% of its electricity from the grid (and the eGrid subregion that it is located within).

In [41]:
## Importing grid region electricity values
egrid2021_data = pd.read_excel('../data/eGRID2021_data.xlsx', sheet_name = "SRL21").clean_names(strip_underscores = True).drop(0)

### Standardizing state names and cleaning up rows in eGrid data

In [42]:
## Cleaning egrid data for usability
emissions_by_subregion = egrid2021_data.copy()

emissions_by_subregion["co2e_lbs_per_mwh"] = pd.to_numeric(emissions_by_subregion["egrid_subregion_annual_co2_equivalent_combustion_output_emission_rate_lb_mwh"])
emissions_by_subregion['subregion'] = emissions_by_subregion['egrid_subregion_acronym']
emissions_by_subregion['subregion_name'] = emissions_by_subregion['egrid_subregion_name']
emissions_by_subregion['co2e_tonnes_per_mwh'] = emissions_by_subregion['co2e_lbs_per_mwh'] / 2204.62262185

# Filtering subregions
excluded_subregions = ["AKGD", "AKMS", "HIMS", "HIOA", "PRMS"]
emissions_by_subregion = emissions_by_subregion[~emissions_by_subregion['subregion'].isin(excluded_subregions)]

# Selecting desired columns
emissions_by_subregion = emissions_by_subregion[['subregion', 'subregion_name', 'co2e_tonnes_per_mwh']]

### Importing GEM data

In [43]:
gem_data_readin = pd.read_excel("../data/GEM_2022_data.xlsx", sheet_name = "Steel Plants").clean_names(strip_underscores = True)

### Filtering to only look at plants and data that we are interested in

In [44]:
## eaf_capacity is in thousand tonnes per year and we are standardizing to tonnes
gem_data_cleaned = gem_data_readin.copy()

## Filtering to the specifications we need
gem_data_cleaned = gem_data_cleaned[
       (gem_data_cleaned['country'] == "United States") &
       (gem_data_cleaned["status"] == "operating") &
       (gem_data_cleaned['nominal_eaf_steel_capacity_ttpa'] != "NaN")
]

## Needed to do this in another step to make sure start_date was properly filtered
gem_data_cleaned = gem_data_cleaned[
       (gem_data_cleaned['start_date'] < 2022) &
       (~gem_data_cleaned['plant_id'].isin(["SUS00009", "SUS00061"]))
]

gem_data_cleaned = gem_data_cleaned.rename(columns={'plant_name_english':'plant_name'
, 'subnational_unit_province_state':'state'
, 'location_address':'address'})

gem_data_cleaned['eaf_capacity'] = pd.to_numeric(gem_data_cleaned['nominal_eaf_steel_capacity_ttpa'])
gem_data_cleaned['max_tonnes_of_steel_producible_annually'] = gem_data_cleaned['eaf_capacity'] * 1000

gem_data_cleaned = gem_data_cleaned[['plant_id'
         , 'plant_name'
         , 'owner'
         , 'coordinates'
         , 'country'
         , 'state'
         , 'status'
         , 'start_date'
         , 'plant_age_years'
         , 'max_tonnes_of_steel_producible_annually'
         , 'municipality'
         , 'address'
         , 'category_steel_product'
         , 'steel_products'
         , 'responsiblesteel_certification']]

gem_data = gem_data_cleaned.drop(columns=['country', 'start_date', 'status', 'responsiblesteel_certification'])

## Separate the "coordinates" column into "lat" and "lon" columns
gem_data[['lat', 'lon']] = gem_data['coordinates'].str.split(',', expand=True)

## Remove the "coordinates" column
gem_data.drop(columns=['coordinates'], inplace=True)

## Reordering columns
gem_data = gem_data[['plant_id', 'plant_name', 'owner', 'lat', 'lon', 'state', 'plant_age_years', 'max_tonnes_of_steel_producible_annually', 'municipality', 'address', 'category_steel_product', 'steel_products']]

In [54]:
# Reading in data
subregion_shapes_raw = gpd.read_file("../data/egrid2020_subregions/eGRID2020_subregions.shp").clean_names()

# Filtering subregion shapes
# Simplifying subregion shapes
subregion_shapes = subregion_shapes_raw[~subregion_shapes_raw['zipsubregi'].isin(["AKGD", "AKMS", "HIMS", "HIOA", "PRMS"])]

''''
subregion_shapes = subregion_shapes.simplify(tolerance=0.0005)


# Bringing in plant points
plant_points = gpd.GeoDataFrame(gem_data, geometry=gpd.points_from_xy(gem_data['lon'], gem_data['lat']))
plant_points.crs = "EPSG:4326"

#### Find out how to properly find overlapping rows
# Checking overlaps
overlap = plant_points.geometry.within(subregion_shapes.unary_union)

# Creating subregion vector
subregion_vector = []
for i in range(len(overlap)):
    if overlap[i]:
        subregion_vector.append(subregion_shapes['zipsubregi'][i])
    else:
        subregion_vector.append(None)

# Adding subregion column to gem_data
gem_data['subregion'] = subregion_vector

# Merging data
plant_emissions_by_subregion = gem_data.merge(emissions_by_subregion, on="subregion", how="left")
'''

'\'\nsubregion_shapes = subregion_shapes.simplify(tolerance=0.0005)\n\n\n# Bringing in plant points\nplant_points = gpd.GeoDataFrame(gem_data, geometry=gpd.points_from_xy(gem_data[\'lon\'], gem_data[\'lat\']))\nplant_points.crs = "EPSG:4326"\n\n#### Find out how to properly find overlapping rows\n# Checking overlaps\noverlap = plant_points.geometry.within(subregion_shapes.unary_union)\n\n# Creating subregion vector\nsubregion_vector = []\nfor i in range(len(overlap)):\n    if overlap[i]:\n        subregion_vector.append(subregion_shapes[\'zipsubregi\'][i])\n    else:\n        subregion_vector.append(None)\n\n# Adding subregion column to gem_data\ngem_data[\'subregion\'] = subregion_vector\n\n# Merging data\nplant_emissions_by_subregion = gem_data.merge(emissions_by_subregion, on="subregion", how="left")\n'

In [55]:
subregion_shapes

,zipsubregi,shape_leng,shape_le_1,shape_area,geometry
2,AZNM,85.346813,85.346813,54.991175,"MULTIPOLYGON (((-104.35820 33.30621, -104.3585..."
3,CAMX,75.567466,75.567466,35.788722,"MULTIPOLYGON (((-117.15658 32.66883, -117.1566..."
4,ERCT,147.354169,147.354169,49.825826,"MULTIPOLYGON (((-97.19964 26.00022, -97.19972 ..."
5,FRCC,147.119699,147.119699,12.279977,"MULTIPOLYGON (((-81.96339 24.52110, -81.96374 ..."
8,MROE,74.366089,74.366089,9.751795,"MULTIPOLYGON (((-87.83248 43.83403, -87.83289 ..."
9,MROW,118.469797,118.469797,109.918613,"MULTIPOLYGON (((-91.42487 40.58896, -91.42515 ..."
10,NEWE,107.279556,107.279556,19.181889,"MULTIPOLYGON (((-73.62286 40.98370, -73.62284 ..."
11,NWPP,207.631377,207.631377,186.094937,"MULTIPOLYGON (((-108.45100 35.52864, -108.4512..."
12,NYCW,9.730216,9.730216,0.201028,"MULTIPOLYGON (((-74.05047 40.56641, -74.05026 ..."
13,NYLI,22.880789,22.880789,0.328409,"MULTIPOLYGON (((-73.61637 40.59684, -73.61620 ..."


In [47]:
plant_points

,plant_id,plant_name,owner,lat,lon,state,plant_age_years,max_tonnes_of_steel_producible_annually,municipality,address,category_steel_product,steel_products,geometry
1349,SUS00016,SSAB Americas Alabama steel plant,SSAB Americas Holding AB,30.941783,-88.012437,Alabama,22,1250000.0,Axis,"12400 Highway 43 North, Axis, Alabama 36505, U...",semi-finished,"heavy plate, coil",POINT (-88.01244 30.94178)
1350,SUS00075,CMC Alabama steel plant,Commercial Metals Co,33.534372,-86.75773,Alabama,29,653000.0,Birmingham,"101 South 50th St., Birmingham, AL 35212, Unit...",finished rolled,equal angles; unequal angles; flats; channels,POINT (-86.75773 33.53437)
1352,SUS00037,Outokumpu Stainless USA steel plant,Outokumpu Stainless Usa LLC,31.148179,-87.988660,Alabama,16,1000000.0,Calvert,"1 Steel Dr., Calvert, Alabama, 36513-1300, Uni...",crude; semi-finished; finished rolled,"stainless, hot-rolled coil, cold-rolled coil, ...",POINT (-87.98866 31.14818)
1353,SUS00048,U.S. Steel Fairfield plant,United States Steel Corp,33.480697,-86.937854,Alabama,3,1600000.0,Fairfield,"5700 Valley Road, Fairfield, Alabama 35064, Un...",semi-finished; finished rolled,"pipe, flat-rolled products, tubular products",POINT (-86.93785 33.48070)
1354,SUS00028,Nucor Steel Decatur plant,Nucor Steel Decatur LLC,34.639630,-87.085738,Alabama,26,2268000.0,Trinity,"4301 Iverson Boulevard, Trinity, Alabama 35673...",finished rolled,sheet; tubular products,POINT (-87.08574 34.63963)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427,SUS00034,Nucor Steel Plymouth plant,Nucor Corp,41.882789,-112.196743,Utah,42,908000.0,Plymouth,"7285 W 21200 N West Cemetery Road, Plymouth, U...",finished rolled,bar,POINT (-112.19674 41.88279)
1428,SUS00015,Gerdau Petersburg Steel Mill,Gerdau Long Steel North America,37.181168,-77.449517,Virginia,25,1000000.0,Petersburg,"25801 Hofheimer Way, Petersburg, Virginia 2380...",semi-finished; finished rolled,channels; H piling; MC channels; sheet pile; w...,POINT (-77.44952 37.18117)
1429,SUS00055,Steel Dynamics Roanoke steel plant,Steel Dynamics Inc,37.272927,-79.998828,Virginia,68,660000.0,Roanoke,"102 Westside Blvd, Roanoke, VA 24017",crude,merchant bars,POINT (-79.99883 37.27293)
1430,SUS00035,Nucor Steel Seattle plant,Nucor Steel Seattle Inc,47.569597,-122.367395,Washington,119,855000.0,Seattle,"2424 SW Andover Street, Seattle, Washington 98...",finished rolled,bar,POINT (-122.36740 47.56960)
